In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import Card, Deck
from pypokerengine.api.game import setup_config, start_poker

import pickle
import tensorflow as tf
import random

import sys
sys.path.insert(0, '../scripts/')

import PlayerModels as pm
from MyEmulator import MyEmulator
from DQNPlayer import DQNPlayer
from util import *

## Initialization

In [2]:
h_size = 128

In [27]:
%time main_wp = DQNPlayer(h_size=h_size, is_restore=True, is_train=False, debug=True, is_double=True)

INFO:tensorflow:Restoring parameters from ../cache/models/DQN/-20000
CPU times: user 816 ms, sys: 76 ms, total: 892 ms
Wall time: 838 ms


## Testing

In [4]:
config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15, summary_file='/dev/null')

config.register_player(name="wp", algorithm=main_wp)
# config.register_player(name="r2", algorithm=RandomPlayer())
config.register_player(name="f2", algorithm=pm.CallPlayer())
config.register_player(name="f3", algorithm=pm.CallPlayer())
config.register_player(name="f4", algorithm=pm.CallPlayer())
config.register_player(name="f5", algorithm=pm.CallPlayer())
config.register_player(name="f6", algorithm=pm.CallPlayer())
config.register_player(name="f7", algorithm=pm.CallPlayer())
config.register_player(name="f8", algorithm=pm.CallPlayer())
config.register_player(name="f9", algorithm=pm.CallPlayer())

game_result = start_poker(config, verbose=1)

(['Hole:', ['D9', 'CA']],)
(['Start stack:', 1500],)
(['Estimation:', 0.1455],)
Started the round 1
Street "preflop" started. (community card = [])
"f4" declared "call:30"
"f5" declared "call:30"
"f6" declared "call:30"
"f7" declared "call:30"
"f8" declared "call:30"
"f9" declared "call:30"
(array([-0.09245121, -2.76841259, -3.08255911, -4.92908478, -5.19628429], dtype=float32),)
"wp" declared "fold:0"
"f2" declared "call:30"
"f3" declared "call:30"
Street "flop" started. (community card = ['S2', 'S9', 'H8'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
Street "turn" started. (community card = ['S2', 'S9', 'H8', 'S3'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
Street "river" started. (community card = ['S2', '

"f2" declared "call:30"
"f3" declared "call:30"
Street "flop" started. (community card = ['CQ', 'SK', 'CJ'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
Street "turn" started. (community card = ['CQ', 'SK', 'CJ', 'S6'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
Street "river" started. (community card = ['CQ', 'SK', 'CJ', 'S6', 'DT'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"['f6']" won the round 10 (stack = {'wp': 1455, 'f2': 1935, 'f3': 1200, 'f4': 1200, 'f5': 1680, 'f6': 1710, 'f7': 1440, 'f8': 1680, 'f9': 1200})
(['End stack:', 1455],)
(['Hole:', ['HA', 'H3']],)
([

"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"['f4']" won the round 20 (stack = {'wp': 1410, 'f2': 1875, 'f3': 1140, 'f4': 1140, 'f5': 2145, 'f6': 1650, 'f7': 1620, 'f8': 1620, 'f9': 900})
(['End stack:', 1410],)
(['Hole:', ['HQ', 'CQ']],)
(['Start stack:', 1410],)
(['Estimation:', 0.2583],)
Started the round 21
Street "preflop" started. (community card = [])
"f6" declared "call:30"
"f7" declared "call:30"
"f8" declared "call:30"
"f9" declared "call:30"
(array([ 0.35592696, -3.07934809, -4.13775635, -2.59501147, -5.30900002], dtype=float32),)
"wp" declared "fold:0"
"f2" declared "call:30"
"f3" declared "call:30"
"f4" declared "call:30"
"f5" declared "call:30"
Street "flop" started. (community card = ['C4', 'H6', 'H9'])
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
Street "turn" started. (community card = ['C4', 'H6', 'H9', 'C2

"f2" declared "call:0"
"f3" declared "call:0"
Street "turn" started. (community card = ['D3', 'D5', 'H4', 'H2'])
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
Street "river" started. (community card = ['D3', 'D5', 'H4', 'H2', 'CQ'])
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
"['f2']" won the round 30 (stack = {'wp': 1365, 'f2': 2055, 'f3': 840, 'f4': 1080, 'f5': 1845, 'f6': 2340, 'f7': 1575, 'f8': 1800, 'f9': 600})
(['End stack:', 1365],)
(['Hole:', ['S4', 'H4']],)
(['Start stack:', 1365],)
(['Estimation:', 0.1324],)
Started the round 31
Street "preflop" started. (community card = [])
"f7" declared "call:30"
"f8" declared "call:30"
"f9" declared "call:30"
(array([-0.6129418 , -3.96111012, -3.66722822, -4.5778989

"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"['f9']" won the round 40 (stack = {'wp': 1320, 'f2': 2025, 'f3': 1020, 'f4': 1020, 'f5': 1905, 'f6': 2040, 'f7': 1650, 'f8': 1740, 'f9': 780})
(['End stack:', 1320],)
(['Hole:', ['HA', 'S9']],)
(['Start stack:', 1320],)
(['Estimation:', 0.1493],)
Started the round 41
Street "preflop" started. (community card = [])
"f8" declared "call:30"
"f9" declared "call:30"
(array([-0.48728395, -2.66678333, -3.49156809, -2.60201192, -4.14161491], dtype=float32),)
"wp" declared "fold:0"
"f2" declared "call:30"
"f3" declared "call:30"
"f4" declared "call:30"
"f5" declared "call:30"
"f6" declared "call:30"
"f7" declared "call:30"
Street "flop" started. (community card = ['C4', 'D9', 'SK'])
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
Street "turn" started. (community card =

"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
Street "turn" started. (community card = ['C5', 'D6', 'S6', 'S8'])
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
Street "river" started. (community card = ['C5', 'D6', 'S6', 'S8', 'ST'])
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"['f2']" won the round 49 (stack = {'wp': 1275, 'f2': 2250, 'f3': 750, 'f4': 1500, 'f5': 1635, 'f6': 1770, 'f7': 1620, 'f8': 1710, 'f9': 990})
(['End stack:', 1275],)
(['Hole:', ['SQ', 'DK']],)
(['Start stack:', 1275],)
(['Estimation:', 0.1814],)
Started the round 50
Street "preflop" started. (community card = [])
"f8" declared "call:30"
"f9" declared "call:30"
(array([ 0.2281962 , -2.85878277, -

## Metric

In [14]:
%time main_wp = DQNPlayer(h_size=h_size, is_restore=True, is_train=False, debug=False, is_double=True)

INFO:tensorflow:Restoring parameters from ../cache/models/DQN/-8000
CPU times: user 828 ms, sys: 112 ms, total: 940 ms
Wall time: 867 ms


In [4]:
# config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15, summary_file='/dev/null')

# config.register_player(name="wp", algorithm=main_wp)
# # config.register_player(name="r2", algorithm=RandomPlayer())
# config.register_player(name="f2", algorithm=pm.CallPlayer())
# config.register_player(name="f3", algorithm=pm.CallPlayer())
# config.register_player(name="f4", algorithm=pm.CallPlayer())
# config.register_player(name="f5", algorithm=pm.CallPlayer())
# config.register_player(name="f6", algorithm=pm.CallPlayer())
# config.register_player(name="f7", algorithm=pm.CallPlayer())
# config.register_player(name="f8", algorithm=pm.CallPlayer())
# config.register_player(name="f9", algorithm=pm.CallPlayer())

In [15]:
config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15, summary_file='/dev/null')

config.register_player(name="wp", algorithm=main_wp)
# config.register_player(name="r2", algorithm=RandomPlayer())
config.register_player(name="CallPlayer1", algorithm=pm.CallPlayer())
config.register_player(name="CallPlayer2", algorithm=pm.CallPlayer())
config.register_player(name="FoldPlayer1", algorithm=pm.FoldPlayer())
config.register_player(name="FoldPlayer2", algorithm=pm.FoldPlayer())
config.register_player(name="HeuristicPlayer1", algorithm=pm.HeuristicPlayer())
config.register_player(name="HeuristicPlayer2", algorithm=pm.HeuristicPlayer())
config.register_player(name="RandomPlayer1", algorithm=pm.RandomPlayer())
config.register_player(name="RandomPlayer2", algorithm=pm.RandomPlayer())

In [16]:
%%time
d = None
for i in range(100):
    game_result = start_poker(config, verbose=0)
    t = pd.DataFrame(game_result['players'])
    t['round'] = i
    if d is None:
        d = t
    else:
        d = pd.concat((d, t))

CPU times: user 22min 27s, sys: 492 ms, total: 22min 27s
Wall time: 22min 25s


### With training only with CallPlayer for 3 hours

In [23]:
d.groupby('name').mean()['stack'].sort_values()

name
RandomPlayer1        847.63
RandomPlayer2        870.24
wp                  1011.58
FoldPlayer1         1106.70
FoldPlayer2         1107.45
HeuristicPlayer2    1257.65
HeuristicPlayer1    1348.60
CallPlayer1         2951.04
CallPlayer2         2987.55
Name: stack, dtype: float64

### With training  with different players for 4 hours

In [8]:
d.groupby('name').mean()['stack'].sort_values()

name
RandomPlayer2        301.11
RandomPlayer1        908.14
FoldPlayer2         1076.40
FoldPlayer1         1077.45
HeuristicPlayer2    1249.55
HeuristicPlayer1    1675.50
wp                  1689.70
CallPlayer1         2550.21
CallPlayer2         2965.17
Name: stack, dtype: float64

### With training  with different players for 8 hours

In [6]:
d.groupby('name').mean()['stack'].sort_values()

name
RandomPlayer1        595.12
RandomPlayer2        719.08
FoldPlayer1         1080.90
FoldPlayer2         1087.95
HeuristicPlayer2    1155.82
HeuristicPlayer1    1365.85
wp                  1451.95
CallPlayer2         2663.48
CallPlayer1         3372.21
Name: stack, dtype: float64

### With training  with different players for 15 hours

In [10]:
d.groupby('name').mean()['stack'].sort_values()

name
RandomPlayer1        706.92
RandomPlayer2        944.37
FoldPlayer1         1076.85
FoldPlayer2         1077.45
HeuristicPlayer2    1270.39
HeuristicPlayer1    1343.57
wp                  2168.95
CallPlayer1         2341.03
CallPlayer2         2560.93
Name: stack, dtype: float64

### With training  with different players for 24 hours

In [13]:
d.groupby('name').mean()['stack'].sort_values()

name
RandomPlayer1        851.21
RandomPlayer2        889.64
FoldPlayer2         1080.45
FoldPlayer1         1092.15
HeuristicPlayer2    1425.16
HeuristicPlayer1    1507.93
wp                  1742.96
CallPlayer2         2131.21
CallPlayer1         2768.93
Name: stack, dtype: float64

### With training  with different players for 26 hours

In [17]:
d.groupby('name').mean()['stack'].sort_values()

name
RandomPlayer2        568.01
FoldPlayer2         1083.38
FoldPlayer1         1087.50
RandomPlayer1       1242.15
HeuristicPlayer1    1398.09
HeuristicPlayer2    1471.35
wp                  1714.81
CallPlayer2         2066.96
CallPlayer1         2859.58
Name: stack, dtype: float64